In [1]:
import dataloader
import HMM
import matplotlib.pyplot as plt
import utils
from ipywidgets import interact, BoundedIntText
from IPython.display import display
import mir_eval

model_path = "/home/max/ET-TI/Masterarbeit/models/hmm_model.pkl"   
model = HMM.load_model(model_path)

dataset_path = "/home/max/ET-TI/Masterarbeit/datasets/beatles/"
dataset = dataloader.BeatlesDataset(dataset_path,'madmom',beat_align=False)
dataset_aligned = dataloader.BeatlesDataset(dataset_path,'madmom',beat_align=True)


Load Chromadata and ground truth annotations 

In [2]:
index = 103
ALIGNED = True
print(f"Song: {dataset_aligned.getTitle(index)}")
if ALIGNED:
    t_chroma,chroma,ref_intervals,ref_labels = dataset_aligned[index]
else:
    t_chroma,chroma,ref_intervals,ref_labels = dataset[index]

Song: beatles_12_Let_It_Be_06_Let_It_Be


Predict chords using a trained Hidden Markov model

In [3]:
est_intervals,est_labels = model.predict(t_chroma,chroma)

# visualize results
def adjust_axes(x,y):
    for text in ax[0].texts:
        text.remove()
    interval=(x,y)
    utils.plotAnnotations(ax[0],ref_intervals,ref_labels,interval)
    utils.plotPredictionResult(ax[0],est_intervals,est_labels,interval)
    for i in range(3):
        ax[i].set_xlim(x,y)
    display(fig)

title = dataset_aligned.getTitle(index)
audiopath,labelpath = dataset_aligned.getFilepaths(index)
interval = (0,t_chroma[-1])
fig,ax = plt.subplots(3,1,figsize=(14,10))
fig.tight_layout(pad=3)
utils.plotChroma(ax[1],chroma,time=interval)
utils.plotAudioWaveform(ax[0],audiopath,time=interval)
_,chroma_unprocessed,_,_ = dataset[index]
utils.plotChroma(ax[2],chroma_unprocessed,time=interval)
plt.close()

interact(adjust_axes,
         x = BoundedIntText(value=0,min=0,max=int(chroma.shape[0]/10),step=1,description='xlim:',disabled=False),
         y = BoundedIntText(value=15,min=0,max=int(chroma.shape[0]/10),step=1,description='ylim:',disabled=False)
)

/home/max/.local/lib/python3.10/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


interactive(children=(BoundedIntText(value=0, description='xlim:', max=243), BoundedIntText(value=15, descript…

<function __main__.adjust_axes(x, y)>

Evaluation using MIREX Score

In [4]:
est_intervals, est_labels = mir_eval.util.adjust_intervals(
            est_intervals, est_labels, ref_intervals.min(),
            ref_intervals.max(), mir_eval.chord.NO_CHORD,
            mir_eval.chord.NO_CHORD)
(intervals,ref_labels,est_labels) = mir_eval.util.merge_labeled_intervals(
    ref_intervals, ref_labels, est_intervals, est_labels)
durations = mir_eval.util.intervals_to_durations(intervals)
comparisons = mir_eval.chord.thirds(ref_labels, est_labels)
score = mir_eval.chord.weighted_accuracy(comparisons, durations)
print(f"{dataset.getTitle(index)}: {round(score*100,2)}%")

beatles_12_Let_It_Be_06_Let_It_Be: 84.48%
